In [ ]:
import sys
sys.path.append('..')
sys.path.append('../..')

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from distributions import BivariateGaussian, BivariateSkewNormal
from bayesbridge.reg_coef_sampler.stepsize_adapter \
    import StepsizeAdapter, RobbinsMonroStepsize, DualAverageStepsizeAdapter

## Define an adaptive RWMH operator and the target distribution.

In [ ]:
def random_walk_MH_step(f, theta0, logp0, prop_sd):
    """
    Params
    ------
    f : function
        Computes the log density of the target density
    prop_sd : scalar or vector
        Proposal standard deviation
    """
    theta = theta0.copy()
    theta += prop_sd * np.random.randn(len(theta0))
    logp = f(theta)
    accept_prob = min(1, np.exp(logp - logp0))
    accepted = accept_prob > np.random.uniform()
    if not accepted:
        theta = theta0
        logp = logp0
    return theta, logp, accept_prob

def adaptive_random_walk_MH(
        f, n_iter, x0, stepsize0, target_accept_prob=.9,
        adapt_method='robbins-monro'
    ):
    
    logp = f(x0)
    x = x0.copy()
    stepsize = stepsize0
    
    # Pre-allocate
    samples = np.zeros((len(x0), n_iter))
    stepsizes = np.zeros(n_iter)
    ave_stepsizes = np.zeros(n_iter)
    
    if adapt_method == 'dual-average':
        adapter = DualAverageStepsizeAdapter(
            stepsize0, target_accept_prob
        )
    else:
        adapter = StepsizeAdapter(
            stepsize0, target_accept_prob, 
            reference_iteration=n_iter, 
            adaptsize_at_reference=0.05
        )
    for i in range(n_iter):
        
        x, logp, accept_prob = random_walk_MH_step(f, x, logp, stepsize)
        stepsize = adapter.adapt_stepsize(accept_prob)
        
        samples[:, i] = x
        stepsizes[i] = stepsize
        ave_stepsizes[i] = adapter.get_current_stepsize(averaged=True)
        
    return samples, stepsizes, ave_stepsizes

In [ ]:
bi_skewnorm = BivariateSkewNormal()
def f(x):
    return bi_skewnorm.compute_logp_and_gradient(x, logp_only=True)[0]

## Compare different adaptation schedule for Robbins-Monro algorithm.

In [ ]:
plt.figure(figsize=(7, 4.5))
plt.rcParams['font.size'] = 18

for decay_exponent in (1., 2 / 3, 1 / 2):
    rm_stepsize = RobbinsMonroStepsize(
        init=1., decay_exponent=decay_exponent,
        reference_iteration=100, size_at_reference=.05)
    adaptation_sizes = rm_stepsize.calculate_stepsize(np.arange(500))
    plt.plot(np.log10(adaptation_sizes), label='Exponent = {:.2f}'.format(decay_exponent))
    
plt.ylabel(r'$\log_{10}$(adaptation size)')
plt.xlabel('Number of adaptation steps')
plt.legend()
plt.show()

## Run an adaptive MCMC with Robbins-Monro adaptation.

In [ ]:
stepsize0 = 1
x0 = np.array([0., 0.])
n_iter = 5 * 10 ** 4

samples, stepsizes, ave_stepsizes = adaptive_random_walk_MH(
   f, n_iter, x0, stepsize0, target_accept_prob=.9, 
    adapt_method='robbins-monro',
)

#### Take a look at the empirical distribution: stationary distribution may be perturbed a bit due to adaptation.

In [ ]:
plt.figure(figsize=(7, 4.5))
plt.rcParams['font.size'] = 20

grid = np.linspace(-4, 4, 101)
marginal_pdf = bi_skewnorm.compute_marginal_pdf(grid, grid)

for axis in range(2):
    color = 'C' + str(axis)
    plt.hist(samples[axis, int(n_iter / 2):], 
             alpha=.5, bins=21, normed=True,
             color=color)
    plt.plot(grid, marginal_pdf[axis], color=color)
plt.show()

#### Plot the sequence of stepsizes used at each MCMC iteration as well as the average.

In [ ]:
plt.figure(figsize=(14, 4.5))

plt.plot(np.log10(stepsizes), label='M-H stepsize')
plt.plot(np.log10(ave_stepsizes), label='averaged stepsize')
plt.ylim([-2.1, -.4])

plt.legend()
plt.show()

## Run an adaptive MCMC with Robbins-Monro adaptation.

In [ ]:
stepsize0 = .1
x0 = np.array([0., 0.])
n_iter = 5 * 10 ** 4
samples, stepsizes, ave_stepsizes = adaptive_random_walk_MH(
    f, n_iter, x0, stepsize0, target_accept_prob=.9, 
    adapt_method='dual-average',
)

#### Take a look at the empirical distribution: stationary distribution may be perturbed a bit due to adaptation.

In [ ]:
plt.figure(figsize=(7, 4.5))
plt.rcParams['font.size'] = 20

grid = np.linspace(-4, 4, 101)
marginal_pdf = bi_skewnorm.compute_marginal_pdf(grid, grid)

for axis in range(2):
    color = 'C' + str(axis)
    plt.hist(samples[axis, int(n_iter / 2):], 
             alpha=.5, bins=21, normed=True,
             color=color)
    plt.plot(grid, marginal_pdf[axis], color=color)
plt.show()

#### Plot the sequence of stepsizes used at each MCMC iteration as well as the average.

In [ ]:
plt.figure(figsize=(14, 4.5))

plt.plot(np.log10(stepsizes), label='M-H stepsize')
plt.plot(np.log10(ave_stepsizes), label='averaged stepsize')
plt.ylim([-2.1, -.4])

plt.legend()
plt.show()